# Clasificación con Árbol de Decisión para Datos de Béisbol

Este notebook implementa un modelo de árbol de decisión para clasificar equipos de béisbol según su rendimiento (alto, medio, bajo) basado en estadísticas de bateo.

## Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

## Cargar los datos

In [ ]:
# Cargar los datos
df = pd.read_csv('../data/beisbol.csv')

# Exploración inicial
print("Información del dataset:")
df.info()

print("\nEstadísticas descriptivas:")
df.describe()

## Crear variable categórica

In [ ]:
# Crear una variable categórica basada en 'runs'
# Clasificaremos los equipos en rendimiento alto, medio o bajo según sus carreras
q_33 = df['runs'].quantile(0.33)
q_66 = df['runs'].quantile(0.66)

def categorize_runs(runs):
    if runs <= q_33:
        return 0  # Rendimiento bajo
    elif runs <= q_66:
        return 1  # Rendimiento medio
    else:
        return 2  # Rendimiento alto

df['rendimiento'] = df['runs'].apply(categorize_runs)

# Verificar la distribución de clases
print("\nDistribución de clases (rendimiento):")
print(df['rendimiento'].value_counts())

# Visualizar la distribución de clases
plt.figure(figsize=(8, 6))
sns.countplot(x='rendimiento', data=df)
plt.title('Distribución de Clases de Rendimiento')
plt.xlabel('Clase de Rendimiento (0=Bajo, 1=Medio, 2=Alto)')
plt.ylabel('Cantidad de Equipos')
plt.savefig('../results/beisbol_distribucion_clases.png')
plt.show()

## Visualización de datos

In [ ]:
# Visualizar la relación entre bateos y rendimiento
plt.figure(figsize=(10, 6))
sns.boxplot(x='rendimiento', y='bateos', data=df)
plt.title('Bateos por Categoría de Rendimiento')
plt.xlabel('Rendimiento (0=Bajo, 1=Medio, 2=Alto)')
plt.ylabel('Bateos')
plt.savefig('../results/beisbol_bateos_por_rendimiento.png')
plt.show()

## Preparación de datos

In [ ]:
# Separar características y variable objetivo
X = df[['bateos']]
y = df['rendimiento']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]} muestras')

## Optimización de hiperparámetros

In [ ]:
# Definir la cuadrícula de hiperparámetros
param_grid = {
    'max_depth': [None, 3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Crear el modelo base
tree = DecisionTreeClassifier(random_state=42)

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entrenar el modelo con búsqueda de cuadrícula
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Obtener el mejor modelo
best_tree = grid_search.best_estimator_

## Evaluación del modelo

In [ ]:
# Realizar predicciones
y_pred = best_tree.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud (Accuracy): {accuracy:.4f}')

# Informe de clasificación
print('
Informe de clasificación:')
print(classification_report(y_test, y_pred, target_names=['Bajo', 'Medio', 'Alto']))

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Bajo', 'Medio', 'Alto'], yticklabels=['Bajo', 'Medio', 'Alto'])
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.savefig('../results/beisbol_tree_confusion_matrix.png')
plt.show()

## Visualización del árbol de decisión

In [ ]:
# Visualizar el árbol de decisión
plt.figure(figsize=(15, 10))
plot_tree(best_tree, filled=True, feature_names=['bateos'], class_names=['Bajo', 'Medio', 'Alto'], rounded=True, fontsize=10)
plt.title('Árbol de Decisión para Clasificación de Rendimiento')
plt.savefig('../results/beisbol_decision_tree.png')
plt.show()

## Guardar el modelo

In [ ]:
# Guardar el modelo entrenado
with open('../models/modelo_beisbol_tree.pkl', 'wb') as archivo:
    pickle.dump(best_tree, archivo)

print('Modelo guardado correctamente.')